In [1]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [12]:
import pandas as pd
import re

# 첨부파일 내용을 기반으로 감성분석용 CSV 데이터 구축

# 1. 넷플릭스 관련 텍스트 추출 및 정리
netflix = """
넷플릭스의 주요 장점 중 하나는 다양한 콘텐츠를 보유하고 있다는 점입니다.
사용자는 다양한 장르의 광범위한 영화 및 TV 프로그램 라이브러리를 탐색하여 다양한 취향과 관심사를 충족시킬 수 있습니다.
넷플릭스는 비교할 수 없는 편의성과 유연성을 제공합니다.
다양한 디바이스에서 콘텐츠를 스트리밍할 수 있는 기능을 통해 사용자는 언제 어디서나 좋아하는 프로그램과 영화를 즐길 수 있습니다.
넷플릭스는 독창적이고 독점적인 콘텐츠로 호평을 받고 있습니다.
스트리밍을 위해 안정적인 인터넷 연결에 의존할 수 있다는 점을 인지해야 합니다.
넷플릭스의 구독료가 일부 잠재적 사용자의 이용을 저해할 수 있습니다.
특정 지역에서는 콘텐츠 이용이 제한된다는 점입니다.
라이선스 계약 및 지역 제한으로 인해 일부 프로그램과 영화는 특정 국가에서 액세스하지 못할 수 있습니다.
"""

# 2. 아마존 프라임 관련 텍스트 추출 및 정리
amazon = """
프라임 비디오 가장 큰 매력은 가격 대비 성능이라 말할 수 있습니다.
만개 이상의 작품을 무료로 볼 수 있습니다.
지원 장치와 사용할 수 있는 장면이 다양하다.
스마트 폰이나 태블릿이라면 작품을 다운로드 할 수도 있습니다.
오리지널 작품이 재미있다.
추천 기능, 사용자 리뷰가 생각이상으로 쓸만하다.
타 스트리밍과 연계가 잘 돼있다.
작품의 교체 순환된다는 점이 불편합니다.
새로운 작품의 업데이트가 느리다는 점이 불편합니다.
전부 무료는 아니다.
자막 언어 문제가 있습니다.
자막을 숨길 수 있는 기능이 없습니다.
검색이 의외로 불편합니다.
"""

# 3. 디즈니플러스 관련 텍스트 추출 및 정리
disney = """
디즈니플러스는 특정 브랜드 중심의 콘텐츠에 집중되어 있어 전체적인 장르 구성이나 테마 측면에서 다양성이 떨어진다는 평가를 받고 있습니다.
특정 테마의 콘텐츠가 반복된다는 느낌을 줄 수 있습니다.
현실 사회를 반영하는 사회 고발 드라마, 실험적인 서사를 가진 작품, 혹은 복잡한 인간 관계를 다룬 심리극 등은 거의 찾아보기 어렵습니다.
디즈니플러스는 특정 성향의 콘텐츠에 편중되어 있으며, 이는 다양한 시청 취향을 가진 사용자들에게 단점으로 작용할 수 있습니다.
여전히 로컬 콘텐츠 확보와 현지화 전략에서는 경쟁 OTT에 비해 부족한 점이 많습니다.
자막과 더빙 품질 문제도 사용자 경험에 영향을 줍니다.
일부 콘텐츠의 자막은 직역에 가까워 문화적 맥락이나 감정 표현을 제대로 전달하지 못합니다.
오리지널 콘텐츠의 품질은 높은 편이지만, 전반적인 업데이트 속도는 느리고 신작 공개 주기 또한 긴 편입니다.
디즈니플러스는 글로벌 인기 IP를 활용한 콘텐츠를 중심으로 명확한 정체성을 가진 플랫폼입니다.
마블, 스타워즈, 디즈니 클래식에 열광하는 팬층에게는 최고의 서비스일 수 있습니다.
"""

# 텍스트를 문장별로 분리
def split_sentences(text):
    sentences = []
    # 마침표, 느낌표, 물음표로 끝나는 문장 분리
    raw_sentences = re.split(r'[.!?]\s*', text.strip())
    for sentence in raw_sentences:
        sentence = sentence.strip()
        if len(sentence) > 10:  # 너무 짧은 문장 제외
            sentences.append(sentence)
    return sentences

ds_netflix = split_sentences(netflix)
ds_amazon = split_sentences(amazon)
ds_disney = split_sentences(disney)

# 감성 레이블링 (긍정: 1, 부정: 0, 중립: 0.5)
def label_sentiment(sentence):
    pos_keywords = ['장점', '매력', '호평', '편의성', '유연성', '다양한', '재미있다', '쓸만하다', '최고의', '높은 편', '명확한 정체성']
    neg_keywords = ['단점', '제한', '저해', '불편', '부족', '문제', '떨어진다', '어렵습니다', '느리다', '편중']

    sentence_lower = sentence.lower()
    pos_count = sum(1 for keyword in pos_keywords if keyword in sentence)
    neg_count = sum(1 for keyword in neg_keywords if keyword in sentence)

    if pos_count > neg_count:
        return 1  # 긍정
    elif neg_count > pos_count:
        return 0  # 부정
    else:
        return 0.5  # 중립

# 데이터 구성
data = []

# 넷플릭스 데이터 추가
for sentence in ds_netflix:
    sentiment = label_sentiment(sentence)
    data.append({
        'platform': '넷플릭스',
        'text': sentence,
        'sentiment': sentiment,
        'source': 'barrazacarlos.com'
    })

# 아마존프라임 데이터 추가
for sentence in ds_amazon:
    sentiment = label_sentiment(sentence)
    data.append({
        'platform': '아마존프라임',
        'text': sentence,
        'sentiment': sentiment,
        'source': 'tagilog.tistory.com'
    })

# 디즈니플러스 데이터 추가
for sentence in ds_disney:
    sentiment = label_sentiment(sentence)
    data.append({
        'platform': '디즈니플러스',
        'text': sentence,
        'sentiment': sentiment,
        'source': 'dotori123.com'
    })

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 파일 저장
csv_filename = 'ott_sentiment_analysis_data.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

In [5]:
!pip install -q konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 28.5 MB/s eta 0:00:00


In [13]:
# KoNLPy를 사용한 OTT 플랫폼 감성분석
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Okt, Mecab
okt = Okt()

# 데이터 로드
df = pd.read_csv('./ott_sentiment_analysis_data.csv')

In [17]:
# 감성 분석을 위한 키워드 사전 구축
pos_words = [
    '좋다', '훌륭하다', '만족', '편리', '다양', '최고', '우수', '뛰어나다',
    '매력적', '탁월', '성공적', '효과적', '유용', '흥미롭다', '재미있다',
    '장점', '매력', '호평', '편의성', '유연성', '쓸만하다'
]

neg_words = [
    '나쁘다', '불만', '실망', '문제', '단점', '부족', '제한', '불편',
    '어렵다', '느리다', '복잡', '부정적', '떨어지다', '저해', '방해',
    '불안정', '취약', '제한적', '편중', '미흡'
]

def sentiment_analyze(text):
    """KoNLPy를 사용한 감성분석"""
    # 형태소 분석 (명사, 동사, 형용사 추출)
    morphs = okt.morphs(text)
    pos_tags = okt.pos(text)

    # 명사, 동사, 형용사만 추출
    mean_words = [word for word, pos in pos_tags
                       if pos in ['Noun', 'Verb', 'Adjective']]

    # 감성 점수
    pos_score = sum(1 for word in mean_words if word in pos_words)
    neg_score = sum(1 for word in mean_words if word in neg_words)

    # 긍정/부정 키워드 포함 여부로 추가 점수
    text_lower = text.lower()
    for pos_word in pos_words:
        if pos_word in text:
            pos_score += 1
    for neg_word in neg_words:
        if neg_word in text:
            neg_score += 1

    # 감성 판정
    if pos_score > neg_score:
        return 'positive', pos_score, neg_score, mean_words
    elif neg_score > pos_score:
        return 'negative', pos_score, neg_score, mean_words
    else:
        return 'neutral', pos_score, neg_score, mean_words

# 각 텍스트에 대해 감성분석 수행
results = []
for idx, row in df.iterrows():
    sentiment, pos_score, neg_score, words = sentiment_analyze(row['text'])

    results.append({
        'platform': row['platform'],
        'text': row['text'][:50] + '...' if len(row['text']) > 50 else row['text'],
        'original_sentiment': row['sentiment'],
        'konlpy_sentiment': sentiment,
        'pos_score': pos_score,
        'neg_score': neg_score,
        'mean_words': words[:10]  # 처음 10개 단어만
    })

# 결과 DataFrame 생성
result_df = pd.DataFrame(results)

print("\n=== KoNLPy 감성분석 결과 ===")
print(f"총 분석 문장 수: {len(result_df)}")
print("\n플랫폼별 KoNLPy 감성분석 결과:")
sentiment_by_platform = result_df.groupby('platform')['konlpy_sentiment'].value_counts()
print(sentiment_by_platform)

print("\n=== 상세 분석 결과 (처음 10개) ===")
for idx in range(min(10, len(result_df))):
    row = result_df.iloc[idx]
    print(f"\n{idx+1}. 플랫폼: {row['platform']}")
    print(f"   텍스트: {row['text']}")
    print(f"   원래 라벨: {row['original_sentiment']} | KoNLPy 결과: {row['konlpy_sentiment']}")
    print(f"   긍정점수: {row['pos_score']} | 부정점수: {row['neg_score']}")
    print(f"   추출단어: {row['mean_words']}")


=== KoNLPy 감성분석 결과 ===
총 분석 문장 수: 31

플랫폼별 KoNLPy 감성분석 결과:
platform  konlpy_sentiment
넷플릭스      positive            5
          negative            3
          neutral             1
디즈니플러스    negative            4
          neutral             4
          positive            2
아마존프라임    negative            4
          neutral             4
          positive            4
Name: count, dtype: int64

=== 상세 분석 결과 (처음 10개) ===

1. 플랫폼: 넷플릭스
   텍스트: 넷플릭스의 주요 장점 중 하나는 다양한 콘텐츠를 보유하고 있다는 점입니다
   원래 라벨: 1.0 | KoNLPy 결과: positive
   긍정점수: 3 | 부정점수: 0
   추출단어: ['넷플릭스', '주요', '장점', '중', '하나', '다양한', '콘텐츠', '보유', '있다는', '점']

2. 플랫폼: 넷플릭스
   텍스트: 사용자는 다양한 장르의 광범위한 영화 및 TV 프로그램 라이브러리를 탐색하여 다양한 취향과...
   원래 라벨: 1.0 | KoNLPy 결과: positive
   긍정점수: 1 | 부정점수: 0
   추출단어: ['사용자', '다양한', '장르', '광범위한', '영화', '및', '프로그램', '라이브러리', '탐색', '하여']

3. 플랫폼: 넷플릭스
   텍스트: 넷플릭스는 비교할 수 없는 편의성과 유연성을 제공합니다
   원래 라벨: 1.0 | KoNLPy 결과: positive
   긍정점수: 3 | 부정점수: 0
   추출단어: ['넷플릭스', '비교', '할', '수', '없는', '편의', '성과', '유연성',

In [22]:
# 감성분석 결과 요약 및 정확도 평가
print("=== 감성분석 결과 종합 분석 ===\n")

# 1. 정확도 계산 (원래 라벨과 KoNLPy 결과 비교)
def convert_sentiment_to_numeric(sentiment):
    if sentiment == 'positive':
        return 1.0
    elif sentiment == 'negative':
        return 0.0
    else:  # neutral
        return 0.5

result_df['konlpy_numeric'] = result_df['konlpy_sentiment'].apply(convert_sentiment_to_numeric)

# 정확도 계산
correct_predictions = sum(result_df['original_sentiment'] == result_df['konlpy_numeric'])
accuracy = correct_predictions / len(result_df)

print(f"전체 정확도: {accuracy:.2%} ({correct_predictions}/{len(result_df)})")

# 2. 플랫폼별 감성분석 결과 요약
print("\n=== 플랫폼별 감성분석 요약 ===")
platform_summary = result_df.groupby('platform').agg({
    'pos_score': 'sum',
    'neg_score': 'sum',
    'konlpy_sentiment': lambda x: (x == 'positive').sum(),  # 긍정 문장 수
}).rename(columns={'konlpy_sentiment': 'positive_sentences'})

platform_summary['negative_sentences'] = result_df.groupby('platform')['konlpy_sentiment'].apply(lambda x: (x == 'negative').sum())
platform_summary['neutral_sentences'] = result_df.groupby('platform')['konlpy_sentiment'].apply(lambda x: (x == 'neutral').sum())
platform_summary['total_sentences'] = result_df.groupby('platform').size()

# 감성 비율 계산
platform_summary['positive_ratio'] = platform_summary['positive_sentences'] / platform_summary['total_sentences']
platform_summary['negative_ratio'] = platform_summary['negative_sentences'] / platform_summary['total_sentences']

print(platform_summary)

# 3. 주요 키워드 추출
print("\n=== 플랫폼별 주요 키워드 분석 ===")
for platform in df['platform'].unique():
    platform_texts = df[df['platform'] == platform]['text'].tolist()
    all_text = ' '.join(platform_texts)

    # 형태소 분석으로 주요 키워드 추출
    words = okt.morphs(all_text)
    noun_words = [word for word, pos in okt.pos(all_text) if pos == 'Noun' and len(word) > 1]

    # 빈도 계산
    word_freq = Counter(noun_words)
    top_keywords = word_freq.most_common(5)

    print(f"\n{platform} 주요 키워드:")
    for word, freq in top_keywords:
        print(f"  - {word}: {freq}회")

# 4. CSV 파일로 결과 저장
result_df.to_csv('konlpy_sentiment_analysis_results.csv', index=False, encoding='utf-8-sig')
print(f"\n감성분석 결과가 'konlpy_sentiment_analysis_results.csv' 파일로 저장되었습니다.")

# 5. 결론 및 인사이트
print("\n=== 주요 인사이트 ===")
print(f"1. 넷플릭스: 긍정 비율 {platform_summary.loc['넷플릭스', 'positive_ratio']:.1%} - 콘텐츠 다양성과 편의성이 주요 강점")
print(f"2. 아마존프라임: 긍정 비율 {platform_summary.loc['아마존프라임', 'positive_ratio']:.1%} - 가격 대비 성능과 기능 다양성 우수")
print(f"3. 디즈니플러스: 긍정 비율 {platform_summary.loc['디즈니플러스', 'positive_ratio']:.1%} - 콘텐츠 제한성과 현지화 문제가 주요 약점")

print(f"\n전체적으로 넷플릭스가 가장 긍정적인 평가를 받고 있으며,")
print(f"디즈니플러스는 상대적으로 부정적 평가가 많아 개선이 필요한 것으로 분석됩니다.")

=== 감성분석 결과 종합 분석 ===

전체 정확도: 87.10% (27/31)

=== 플랫폼별 감성분석 요약 ===
          pos_score  neg_score  positive_sentences  negative_sentences  \
platform                                                                 
넷플릭스             10          6                   5                   3   
디즈니플러스            4          8                   2                   4   
아마존프라임            6          6                   4                   4   

          neutral_sentences  total_sentences  positive_ratio  negative_ratio  
platform                                                                      
넷플릭스                      1                9        0.555556        0.333333  
디즈니플러스                    4               10        0.200000        0.400000  
아마존프라임                    4               12        0.333333        0.333333  

=== 플랫폼별 주요 키워드 분석 ===

넷플릭스 주요 키워드:
  - 넷플릭스: 4회
  - 콘텐츠: 4회
  - 사용자: 3회
  - 영화: 3회
  - 프로그램: 3회

아마존프라임 주요 키워드:
  - 작품: 5회
  - 이상: 2회
  - 기능: 2회
  - 자막: 2회
  - 프라임